In [ ]:
# IceCream Sales Prediction - Azure ML Notebook

#Configurar o Workspace do Azure ML
if definir_workspace:
    from azureml.core import Workspace
    ws = Workspace.from_config()  # precisa de config.json (opcional, se estiver usando Notebook em AML Studio)
    print("Workspace carregado com sucesso")
    

In [ ]:
#Imports essenciais
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn


In [ ]:
#Definir MLflow para rastrear os experimentos no Azure
mlflow.set_tracking_uri("azureml")
mlflow.set_experiment("IceCreamSalesPrediction")


In [ ]:
#Carregar os dados
df = pd.read_csv("dataset.csv")
df.head()


In [ ]:
#Análise exploratória simples
plt.scatter(df["temperatura"], df["vendas"])
plt.xlabel("Temperatura")
plt.ylabel("Vendas")
plt.title("Temperatura vs Vendas de Sorvete")
plt.show()


In [ ]:
#Separar os dados para treino/teste
X = df[["temperatura"]]
y = df["vendas"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#Treinar o modelo com rastreamento do MLflow
with mlflow.start_run():
    model = LinearRegression()
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)

    # Salvar o modelo
    mlflow.sklearn.log_model(model, "model")
    print(f"Modelo treinado. R2: {r2:.2f}, MSE: {mse:.2f}")
    

In [ ]:
#Registrar modelo no Azure (opcional, automático se usar mlflow)
# from azureml.core.model import Model
# model_path = "model"  # Caminho gerado pelo MLflow
# model = Model.register(workspace=ws, model_path=model_path, model_name="icecream-regressor")